In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [2]:
# Read the draft_2008-2018.csv file from the Resources folder into a Pandas DataFrame
draft_data_df = pd.read_csv(
    Path("./Resources/college/players.csv")
)

# Review the DataFrame
draft_data_df.head()

,ID,active_from,active_to,birth_date,college,height (in),name,position,weight,NBA__3ptapg,...,NCAA_ppg,NBA Games Played,fg%,ppg,effective pct,height/weight,3pt %,Success1,In HoF,Success2
0,1569,2001,2003,12-Feb-78,Indiana University,73,A.J. Guyton,G,180,2.4,...,16.4,0.005682,0.0754,0.054817,0.061333,0.081111,0.0378,0.316144,Not in HoF,N
1,1610,2017,2017,27-Aug-92,Purdue University,85,A.J. Hammons,C,260,0.5,...,12.1,0.001563,0.0810,0.021927,0.061867,0.065385,0.0500,0.281741,Not in HoF,N
2,3280,2010,2015,7-Oct-86,University of Connecticut,74,A.J. Price,G,181,2.6,...,13.0,0.018537,0.0760,0.057807,0.060800,0.081768,0.0316,0.326512,Not in HoF,N
3,4537,1991,1991,11-Sep-64,"University of Massachusetts Amherst, Fairfield...",74,A.J. Wynder,G,180,0.2,...,11.1,0.000426,0.0500,0.019934,0.033333,0.082222,0.0000,0.185915,Not in HoF,N
4,475,2008,2018,14-Jan-85,University of Oregon,73,Aaron Brooks,G,161,3.4,...,13.1,0.045739,0.0826,0.096678,0.065067,0.090683,0.0370,0.417766,Not in HoF,N


In [3]:
# Drop columns unnecessary for this model
draft_data_df = draft_data_df.drop(columns=["name", "active_from", "active_to", "birth_date", "In HoF", "Success2"])
draft_data_df.head()

,ID,college,height (in),position,weight,NBA__3ptapg,NBA__3ptpct,NBA__3ptpg,NBA_efgpct,NBA_fg%,...,NCAA_ftpg,NCAA_games,NCAA_ppg,NBA Games Played,fg%,ppg,effective pct,height/weight,3pt %,Success1
0,1569,Indiana University,73,G,180,2.4,0.378,0.9,0.460,0.377,...,2.5,128,16.4,0.005682,0.0754,0.054817,0.061333,0.081111,0.0378,0.316144
1,1610,Purdue University,85,C,260,0.5,0.500,0.2,0.464,0.405,...,2.7,132,12.1,0.001563,0.0810,0.021927,0.061867,0.065385,0.0500,0.281741
2,3280,University of Connecticut,74,G,181,2.6,0.316,0.8,0.456,0.380,...,2.7,99,13.0,0.018537,0.0760,0.057807,0.060800,0.081768,0.0316,0.326512
3,4537,"University of Massachusetts Amherst, Fairfield...",74,G,180,0.2,0.000,0.0,0.250,0.250,...,2.8,118,11.1,0.000426,0.0500,0.019934,0.033333,0.082222,0.0000,0.185915
4,475,University of Oregon,73,G,161,3.4,0.370,1.3,0.488,0.413,...,2.7,115,13.1,0.045739,0.0826,0.096678,0.065067,0.090683,0.0370,0.417766


In [4]:
draft_data_df.dtypes

ID                    int64
college              object
height (in)           int64
position             object
weight                int64
NBA__3ptapg         float64
NBA__3ptpct         float64
NBA__3ptpg          float64
NBA_efgpct          float64
NBA_fg%             float64
NBA_fg_per_game     float64
NBA_fga_per_game    float64
NBA_ft%             float64
NBA_ft_per_g        float64
NBA_fta_p_g         float64
NBA_g_played          int64
NBA_ppg             float64
NCAA__3ptapg        float64
NCAA__3ptpct        float64
NCAA__3ptpg         float64
NCAA_efgpct         float64
NCAA_fgapg          float64
NCAA_fgpct          float64
NCAA_fgpg           float64
NCAA_ft             float64
NCAA_ftapg          float64
NCAA_ftpg           float64
NCAA_games            int64
NCAA_ppg            float64
NBA Games Played    float64
fg%                 float64
ppg                 float64
effective pct       float64
height/weight       float64
3pt %               float64
Success1            

In [5]:
# Create a list of categorical variables 
categorical_variables = ["college", "position"]

# Display the categorical variables list
categorical_variables


['college', 'position']

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [7]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(draft_data_df[categorical_variables])

In [8]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names(categorical_variables)
)

# Review the DataFrame
encoded_df

,"college_Alcorn State University, Jackson State University",college_Arizona State University,college_Auburn University,"college_Auburn University, Florida State University","college_Auburn University, Georgia Institute of Technology",college_Austin Peay State University,college_Ball State University,"college_Ball State University, Oakland University",college_Baylor University,"college_Baylor University, Oklahoma State University",...,"college_Winston-Salem State University, Oral Roberts University",college_Wright State University,college_Xavier University,position_C,position_C-F,position_F,position_F-C,position_F-G,position_G,position_G-F
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_values_df = draft_data_df.drop(columns=categorical_variables)
encoded_df = pd.concat([encoded_df, numerical_values_df], axis=1)

# Review the Dataframe
encoded_df

,"college_Alcorn State University, Jackson State University",college_Arizona State University,college_Auburn University,"college_Auburn University, Florida State University","college_Auburn University, Georgia Institute of Technology",college_Austin Peay State University,college_Ball State University,"college_Ball State University, Oakland University",college_Baylor University,"college_Baylor University, Oklahoma State University",...,NCAA_ftpg,NCAA_games,NCAA_ppg,NBA Games Played,fg%,ppg,effective pct,height/weight,3pt %,Success1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,128,16.4,0.005682,0.0754,0.054817,0.061333,0.081111,0.0378,0.316144
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.7,132,12.1,0.001563,0.0810,0.021927,0.061867,0.065385,0.0500,0.281741
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.7,99,13.0,0.018537,0.0760,0.057807,0.060800,0.081768,0.0316,0.326512
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.8,118,11.1,0.000426,0.0500,0.019934,0.033333,0.082222,0.0000,0.185915
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.7,115,13.1,0.045739,0.0826,0.096678,0.065067,0.090683,0.0370,0.417766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.8,45,15.5,0.013281,0.0714,0.034884,0.060000,0.070222,0.0301,0.279887
1350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.2,137,14.5,0.009020,0.0838,0.101661,0.067867,0.081111,0.0377,0.381159
1351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.7,122,14.6,0.001705,0.0816,0.049834,0.056000,0.078534,0.0300,0.297672
1352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.8,39,10.0,0.004545,0.0802,0.044850,0.061467,0.073276,0.0312,0.295538


In [10]:
# Fill NBA_ft% null values with league average
encoded_df["NBA_ft%"].fillna(0.7326, inplace=True)

In [12]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df["Success1"]

# Display a sample of y
y[:5]

0    0.316144
1    0.281741
2    0.326512
3    0.185915
4    0.417766
Name: Success1, dtype: float64

In [13]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=["Success1"])

# Review the features DataFrame
X[:5]

,"college_Alcorn State University, Jackson State University",college_Arizona State University,college_Auburn University,"college_Auburn University, Florida State University","college_Auburn University, Georgia Institute of Technology",college_Austin Peay State University,college_Ball State University,"college_Ball State University, Oakland University",college_Baylor University,"college_Baylor University, Oklahoma State University",...,NCAA_ftapg,NCAA_ftpg,NCAA_games,NCAA_ppg,NBA Games Played,fg%,ppg,effective pct,height/weight,3pt %
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.2,2.5,128,16.4,0.005682,0.0754,0.054817,0.061333,0.081111,0.0378
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,2.7,132,12.1,0.001563,0.0810,0.021927,0.061867,0.065385,0.0500
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.7,2.7,99,13.0,0.018537,0.0760,0.057807,0.060800,0.081768,0.0316
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.3,2.8,118,11.1,0.000426,0.0500,0.019934,0.033333,0.082222,0.0000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.2,2.7,115,13.1,0.045739,0.0826,0.096678,0.065067,0.090683,0.0370


In [14]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

369

In [17]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [18]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  number_input_features

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

369

In [19]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  (hidden_nodes_layer1 + 1) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

185

In [20]:
# Create the Sequential model instance
nn = Sequential()

In [21]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [22]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [23]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="linear"))

In [24]:
# Compile the Sequential model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

In [25]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
32/32 [==============================] - 0s 1ms/step - loss: 0.2664 - mse: 0.2664
Epoch 2/50
32/32 [==============================] - 0s 908us/step - loss: 0.1746 - mse: 0.1746
Epoch 3/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0404 - mse: 0.0404
Epoch 4/50
32/32 [==============================] - 0s 933us/step - loss: 0.0148 - mse: 0.0148
Epoch 5/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0067 - mse: 0.0067
Epoch 6/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0046 - mse: 0.0046
Epoch 7/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0036 - mse: 0.0036    
Epoch 8/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0032 - mse: 0.0032
Epoch 9/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0035 - mse: 0.0035    
Epoch 10/50
32/32 [==============================] - 0s 901us/step - loss: 0.0039 - mse: 0.0039  
Epoch 11/50
32/32 [==============================] - 

In [26]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

11/11 - 0s - loss: 0.0064 - mse: 0.0064
Loss: 0.006409005727618933, Accuracy: 0.006409005727618933
